## NLP. Обработка естественного языка

В данной работе выполняется:
1) обработка текста на основе правил
2) интеграция языковой разметки spacy с моделью машинного обучения
3) распознавание эмоциональной составляющей слов и предложений
4) параллелизм подсчета слов с использованием MapReduce

#### 1. NER. Пользовательская разметка именованных сущностей на основе правил

Библиотека Python spaCy предлагает несколько различных методов для выполнения NER на основе правил. Одним из таких методов является EntityRuler.
EntityRuler - это фабрика spaCy, которая позволяет создавать набор шаблонов с соответствующими метками. Фабрика в spaCy - это набор классов и функций, предварительно загруженных в spaCy, которые выполняют определенные задачи. В случае с EntityRuler фабрика позволяет пользователю создать EntityRuler, дать ему набор инструкций, а затем использовать эти инструкции для поиска и маркировки сущностей.

После того как пользователь создал EntityRuler и задал ему набор инструкций, он может добавить его в конвейер spaCy как новую pipe.

pipe - это компонент конвейера. Цель конвейера - принимать входные данные, выполнять над ними какие-то операции, а затем выводить эти операции либо в виде новых данных, либо в виде извлеченных метаданных. pipe - это отдельный компонент. В случае со spaCy есть несколько различных конвейеров, которые выполняют разные задачи. Токенизатор разбивает текст на отдельные лексемы, синтаксический анализатор разбирает текст, а NER идентифицирует сущности и присваивает им соответствующие метки.


Важно помнить, что конвейеры являются последовательными. Это означает, что компоненты, расположенные раньше в конвейере, влияют на то, что получают последующие компоненты. Иногда эта последовательность очень важна, то есть последующие компоненты зависят от предыдущих.

В этом блокноте мы подробно рассмотрим EntityRuler как компонент конвейера модели spaCy. Готовые модели spaCy поставляются с предварительно загруженной моделью NER; однако они не поставляются с EntityRuler. Чтобы включить EntityRuler в модель spaCy, его нужно создать как новую трубу, дать инструкции, а затем добавить в модель. После этого пользователь может сохранить новую модель с EntityRuler на диск.

In [20]:
#импортируйте библиотеку spacy (см.лаб 2)
import spacy


In [21]:
#загрузите корпус русского языка в переменную nlp (см.лаб 2) 

nlp = spacy.load("ru_core_news_md") 

In [22]:
#создайте EntityRuler
ruler = nlp.add_pipe("entity_ruler")

In [5]:
#задайте сущности(категории) и значения
#через следующий код: 
#patterns = [
#                {"label": "...", "pattern": "..."}
#            ]

#где label - название сущности, pattern - значение
# одна строка соответстивует одной сущности и одному значению, чтобы добавить следующую сущность и значение, воспользуйтесь запятой:
#patterns = [
#                {"label": "...", "pattern": "..."},
#                {"label": "...", "pattern": "..."},
#            ]
# задайте сущность "SEASON" и значение "зима", задайте сущность "PROFESSION" и значение "журналисты", задайте сущность "PROFESSION" и значение "исследователи"





In [23]:
patterns = [
    {'label': 'SEASON', 'pattern': 'зима'},
    {'label': 'PROFESSION', 'pattern': 'журналисты'},
    {'label': 'PROFESSION', 'pattern': 'исследователи'},
]

In [24]:
#добавьте описанные выше правила в обработчик nlp:
ruler.add_patterns(patterns)

In [26]:
#далее повторите шаги 1-4 из лаб 2:

aiwinter = open('ЛР3/aiwinter.txt', encoding='utf8').read().replace('\n', '').lower()
doc = nlp(aiwinter)
doc

кажется, что сейчас мы переживаем «золотой век» искусственного интеллекта. компании инвестируют десятки миллиардов долларов в разработку инструментов на базе генеративного ии. журналисты и исследователи экспериментируют с chatgpt, прося его то сочинить стих, то написать статью. правительства многих стран обсуждают или уже принимают законы, регулирующие использование алгоритмов. однако в истории уже были похожие периоды эйфории по поводу искусственного интеллекта. и заканчивались они разочарованием, оттоком инвестиций и замедлением развития технологий. рассказываем, почему происходили эти спады и стоит ли ожидать схожий итог у нынешней ии-лихорадки.«зима искусственного интеллекта» — период, когда происходит снижение финансирования и интереса к исследованиям в области ии. термин впервые появился в 1984 году как тема публичной дискуссии на ежегодной встрече американской ассоциации искусственного интеллекта (aaai). на ней ведущие исследователи в области ии роджер шэнк и марвин мински выска

In [27]:
# найдите именованные сущности документа , выведите значение и сущность через ent.text, ent.label_ (см. в помощь лаб 2)

for token in doc.ents:
    print(token.text, token.label_)

журналисты PROFESSION
исследователи PROFESSION
chatgpt ORG
ии LOC
исследователи PROFESSION
ии роджер шэнк PER
марвин мински PER
ии PER
сми ORG
зима SEASON
общественности.в ORG
джон маккарти PER
джорджтауне LOC
ibm ORG
сми ORG
сша LOC
министерства обороны ORG
сша (darpa) LOC
исследователи PROFESSION
ии PER
deepl ORG
марвина мински PER
сеймура пейперти PER
мала PER
майкла мэнсфилда PER
джеймса лайтхилла PER
целом.в ORG


In [28]:
# добавьте в patterns правило обнаружения года
# для этого составьте правило (обновите значения переменной patterns), глядя по аналогии на пример описания корпусов кампусов формата 324-444 , 672-531:                 
#                {"label": "HOUSES", "pattern": [{"SHAPE": "ddd"}, {"ORTH": "-"}, {"SHAPE": "ddd"}]}

patterns.append({"label": "HOUSES", "pattern": [{"SHAPE": "dddd"}]})
ruler.add_patterns(patterns)
doc = nlp(aiwinter)


In [29]:
# добавьте описанные выше правила в обработчик nlp, примените языковую модель (шаг 4 из лаб 2)
# найдите именованные сущности документа, должны распознаться года

for token in doc.ents:
    print(token.text, token.label_)


журналисты PROFESSION
исследователи PROFESSION
chatgpt ORG
ии LOC
1984 HOUSES
исследователи PROFESSION
ии роджер шэнк PER
марвин мински PER
ии PER
сми ORG
зима SEASON
1974 HOUSES
1980 HOUSES
общественности.в ORG
1956 HOUSES
джон маккарти PER
джорджтауне LOC
ibm ORG
сми ORG
сша LOC
министерства обороны ORG
сша (darpa) LOC
исследователи PROFESSION
ии PER
deepl ORG
1969 HOUSES
марвина мински PER
сеймура пейперти PER
мала PER
майкла мэнсфилда PER
1969 HOUSES
джеймса лайтхилла PER
1973 HOUSES
целом.в ORG


#### 2. Sentiment Analysis Обучение модели

Это процесс анализа текста с целью определения, выражает ли он положительное, отрицательное или нейтральное настроение.

Один из самых простых и старых подходов к анализу настроений — использовать набор предопределенных правил (подобно тому, как мы описывали правила для NER выше) и лексиконов для назначения баллов полярности словам или фразам. Например, модель на основе правил может назначать положительный балл таким словам, как «любовь», «счастливый» или «удивительный», и отрицательный балл таким словам, как «ненависть», «грустный» или «ужасный». Затем модель будет объединять баллы слов в тексте, чтобы определить его общую тональность. Модели на основе правил легко реализовать и интерпретировать, но у них есть несколько серьезных недостатков. Они не способны улавливать контекст, сарказм или нюансы языка, и они требуют много ручных усилий для создания и поддержания правил и лексиконов.

Другой подход к анализу настроений заключается в использовании моделей машинного обучения, которые представляют собой алгоритмы, обучающиеся на данных и делающие прогнозы на основе шаблонов и признаков. Модели машинного обучения могут быть как с учителем, так и без учителя, в зависимости от того, используют ли они маркированные или немаркированные данные для обучения. Модели машинного обучения с учителем, такие как логистическая регрессия, метод опорных векторов или нейронные сети, учатся классифицировать тексты по предопределенным категориям, таким как положительные, отрицательные или нейтральные, на основе маркированных примеров. Модели машинного обучения без учителя, такие как кластеризация, тематическое моделирование или встраивание слов, учатся обнаруживать скрытую структуру и значение текстов на основе немаркированных данных. Модели машинного обучения более гибкие и мощные, чем модели на основе правил, но у них также есть некоторые проблемы. Они требуют большого объема данных и вычислительных ресурсов, они могут быть предвзятыми или неточными из-за качества данных или выбора признаков, и их может быть трудно объяснить или понять.

Прежде, чем выполнять вычисления, необходимо слова представить в векторной форме. Цель вектора слов — заставить вычислительную систему понять слово. Вычислительные системы не могут эффективно понимать текст. Однако они могут быстро и эффективо обрабатывать числа. По этой причине важно преобразовать слово в число.

В данном задании вам необходимо создать модель ML и обучить её распознавать тональность слова, предложения.

In [30]:
# импортируйте pandas
import pandas as pd


In [32]:
# загрузите в dataframe файл sentiment.csv, разделитель ";" , encoding = 'utf8'
# дадайте имена столбцам: 'topic', 'label'

df = pd.read_csv('ЛР3/sentiment.csv',
                 sep=';',
                 encoding='utf8',
                 names=['topic', 'label'])
df.iloc[0:1, :]

,topic,label
0,не оказало,негатив


Файл содержит разметку слов на положительные и отрицательные слова.

In [33]:
# импортируйте из библиотеки sklearn модуль preprocessing 
# подключите функцию LabelEncoder() 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


Label Encoding - это техника, которая используется для преобразования категориальных столбцов в числовые, чтобы их можно было использовать в моделях машинного обучения, которые принимают только числовые данные. Это важный этап предварительной обработки в проекте машинного обучения.

In [34]:
# создайте дополнительный столбец 'label_num' и запишите туда данные столбца 'label', к которым был применен LabelEncoder() 
# таким образом будут столбцы: 'topic', 'label', 'label_num'
# le.fit(df['label'])
df['label_num'] = le.fit_transform(df['label'])
df.iloc[0:1, :]

,topic,label,label_num
0,не оказало,негатив,0


In [35]:
# трансформируйте слова столбца 'topic' в векторную форму и запишите результат в новый столбец 'vector'
# для трансформации в векторную форму пользуйтесь имеющимся конвейером nlp от spacy, пример вызова : nlp(text).vector
# подсказка: простым способом трансформации будет использование lambda - функции к столбцу 'topic'

df['vector'] = df['topic'].apply(lambda x: nlp(x).vector)
df.iloc[0:1, :]

,topic,label,label_num,vector
0,не оказало,негатив,0,"[0.08277767, 0.024399191, 0.28569317, 0.305388..."


In [36]:
#импортируйте train_test_split из sklearn.model_selection 
from sklearn.model_selection import train_test_split


Разделите на обучающие и тестовые выборки :

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

где 

X - столбец значений векторов вашего датафрейма, т.е. имя_датафрейма.vector.values,

Y - столбец label_num вашего датафрейма

test_size - 20%, то есть 80% датафрейма берется на обучение, 20% на тестирование

random_state установите 2022

https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.train_test_split.html

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df['vector'].values,
                                                    df['label_num'].values,
                                                    test_size=0.2,
                                                    random_state=2022)


 #### 2.1 Naive Bayes sentiment analysis

Алгоритм Naive Bayes - это метод классификации, основанный на теореме Байеса. Он используется для предсказания класса наблюдения по набору признаков. Алгоритм Naive Bayes считается одним из самых простых и мощных алгоритмов машинного обучения.

Алгоритм Naive Bayes основан на концепции условной вероятности. В нем делается предположение, что наличие определенного признака в классе не связано с наличием других признаков в этом классе. Это предположение называется условной независимостью класса.

Алгоритм Naive Bayes можно применить к любой задаче классификации, когда у нас есть набор признаков и мы хотим предсказать метку класса. Некоторые распространенные примеры:

Фильтрация спама: У нас есть набор электронных писем, каждое из которых представлено набором слов (признаков), и мы хотим классифицировать их как спам или не спам.

Классификация документов: У нас есть набор документов, каждый из которых представлен набором слов (признаков), и мы хотим классифицировать их по различным категориям (например, спорт, политика, технологии и т. д.).

Анализ настроений: У нас есть набор рецензий на фильмы, каждая из которых представлена набором слов (признаков), и мы хотим классифицировать их как положительные или отрицательные.

Идентификация автора: Имеется набор письменных текстов, каждый из которых представлен словом (признаками), и задача состоит в том, чтобы определить вероятного автора из множества потенциальных авторов. Для этого необходимо проанализировать стиль письма и языковые особенности, чтобы приписать конкретные тексты их авторам.

In [44]:
#необходимо конвертиовать в двумерный массив перед обучением модели, так как сейчас в X_train каждый элемент был отдельным numpy массивом

import numpy as np

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

Алгоритм классификации Multinomial Naive Bayes, как правило, является базовым решением для задачи анализа настроений. Основная идея метода Naive Bayes заключается в том, чтобы найти вероятности классов, присвоенных текстам, используя совместные вероятности слов и классов.

Мультиномиальный NB - это еще один вариант классификатора NB, который часто используется в задачах классификации текстов, включая SA. Он особенно подходит для признаков, представляющих собой частоты или количества слов, которые обычно получают с помощью таких методов, как модель мешка слов или TF-IDF.Мультиномиальный NB-классификатор предполагает, что признаки условно независимы друг от друга, учитывая метку класса, и следуют мультиномиальному распределению. Он рассчитывает вероятность принадлежности документа к определенному классу на основе частот или количества признаков (слов) в документе.

Так как массив содержит отрицательные значения, модель не обучится. Трансформируем в положительные начения, используя:

In [45]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_X_train_2d = scaler.fit_transform(X_train_2d)
scaled_X_test_2d = scaler.transform(X_test_2d)


In [47]:
#импортируйте MultinomialNB из sklearn
from sklearn.naive_bayes import MultinomialNB


In [48]:
# создайте мультиномиальный классификатор Naive Bayes, т.е. создайте экземпляр модели MultinomialNB() в переменную
MNB = MultinomialNB()



https://scikit-learn.org/dev/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [49]:
# обучите классификатор с пом. функции fit, на вход функции передайте scaled_X_train_2d и y_train
MNB.fit(scaled_X_train_2d, y_train)


MultinomialNB()

In [50]:
y_pred = MNB.predict(scaled_X_test_2d)

Вызовите функцию https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.classification_report.html

In [51]:
# передайте в функцию y_test, y_pred
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.50      0.67         2

    accuracy                           0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5



Модель обучена. протестируйте её с помощью функции predict и пеердачи на вход этой функции scaled_test_embed

https://scikit-learn.org/dev/modules/generated/sklearn.naive_bayes.MultinomialNB.html

Определите тональность слов с помощью обученной модели:

1) пессимист
2) разочаровал
3) выберите несколько предложений из файла

!!! не забудьте :
1) трансформировать слова/предложения в вектор
2) применить np.stack
3) при выполнении функции выполнить .reshape(1, -1) к значению после п.2
4) вызвать функцию .predict

In [52]:
my_test1 = np.stack(nlp('пессимист').vector).reshape(1, -1)
my_test2 = np.stack(nlp('разочаровал').vector).reshape(1, -1)
my_test3 = np.stack(nlp('Впрочем, вернемся немного назад.Первая зима ИИ продлилась с 1974 по 1980 год.'.replace('\n', '').lower()).vector).reshape(1, -1)
my_test4 = np.stack(nlp('В нем английский математик дал крайне пессимистический прогноз насчет перспектив ИИ, отдельно отметив, что ни одно из открытий в этой области не оказало существенного влияния на науку и общество в целом.'.replace('\n', '').lower()).vector).reshape(1, -1)

my_test5 = np.stack(nlp('потенциалом').vector).reshape(1, -1)
# 3) выберите несколько предложений из файла


print(MNB.predict(my_test1))
print(MNB.predict(my_test2))
print(MNB.predict(my_test3))
print(MNB.predict(my_test4))
print(MNB.predict(my_test5))

[0]
[0]
[1]
[1]
[1]


 #### 2.2 Обучите модель распознавать тональность

1) самостоятельно выберите предложения для обучения
2) сделайте разметку предложений на позитивные, негативные, нейтральные
3) обучите модель распознавать тональность на основе MultinominalNB (выполните те же шаги, что и выше)

In [38]:
sentences = [sent.text for sent in doc.sents]

In [39]:
df_sent = pd.read_csv('ЛР3/my_sentiment.csv', encoding='utf8', sep=';', names=['topic', 'label'])
df_sent.head(3)

,topic,label
0,они сравнили этот процесс с ядерной зимой — в ...,негатив
1,и в конце концов исследования окажутся полност...,негатив
2,"кажется, что сейчас мы переживаем «золотой век...",позитив


In [40]:
le_sent = preprocessing.LabelEncoder()

In [41]:
df_sent['label_num'] = le_sent.fit_transform(df_sent['label'])
df_sent.head(5)

,topic,label,label_num
0,они сравнили этот процесс с ядерной зимой — в ...,негатив,0
1,и в конце концов исследования окажутся полност...,негатив,0
2,"кажется, что сейчас мы переживаем «золотой век...",позитив,2
3,компании инвестируют десятки миллиардов доллар...,позитив,2
4,журналисты и исследователи экспериментируют с ...,позитив,2


In [42]:
df_sent['vector'] = df_sent['topic'].apply(lambda x: nlp(x).vector)
df_sent.iloc[0:1, :]

,topic,label,label_num,vector
0,они сравнили этот процесс с ядерной зимой — в ...,негатив,0,"[0.026646322, -0.09245006, 0.06024115, -0.0310..."


In [53]:
x_train, x_test, Y_train, Y_test = train_test_split(df_sent['vector'].values,
                                                    df_sent['label_num'].values,
                                                    test_size=0.3,
                                                    random_state=None
                                                    )


x_train_2d = np.stack(x_train)
x_test_2d = np.stack(x_test)


scaler_sent = MinMaxScaler()
scaled_x_train_2d = scaler_sent.fit_transform(x_train_2d)
scaled_x_test_2d = scaler_sent.transform(x_test_2d)


MNB_sent = MultinomialNB()
MNB_sent.fit(scaled_x_train_2d, Y_train)
Y_pred = MNB.predict(scaled_x_test_2d)
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.43      0.60      0.50         5
           1       0.33      0.50      0.40         2
           2       0.00      0.00      0.00         3

    accuracy                           0.40        10
   macro avg       0.25      0.37      0.30        10
weighted avg       0.28      0.40      0.33        10



/opt/homebrew/anaconda3/envs/ds/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ds/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/anaconda3/envs/ds/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

#### 3. MapReduce

От современных приложений добычи данных, которые часто называют анализом "больших данных", требуется способность быстро обрабатывать огромные массивы данных. Во многих таких приложениях структура данных регулярна и существует масса возможностей распараллеливания. 

Одна из возможной - технология MapReduce

Процесс выглядит следующим образом:
1) есть несколько задач Map(распределителей), каждая из которых получает одно или несколько порций файла из распределенной файловой системы. Распределители преобразуют порцию в последовательность пар ключ-значение. Как именно из входных данных порождаются эти пары, определяет функция Map, написанная пользователем. Такие распредлители находятся на разных вычислительных узлах. Один Map на один вычислительный узел/выч.машину.
2) Пары ключ-значение от каждого распределителя собираются главным контроллером и сортируются по ключу. Затем ключи раздаются задачам Reduce(редукторам), так что все пары с одинаковым ключом попадают одному и тому же редуктору.
3) Редукторы обрабатывают по одному ключу за раз и таким образом комбинируют значения, ассоциированые с этим ключом. Способ комбинирования определяется функцией Reduce, написанной пользователем.

 #### 3.1 Реализация функции Map

В этой части мы создадим набор пар ключ-значение и соберем все пары с одинаковым ключом.

В качестве исходных данных здесь используется файл «aiwinter.txt», который мы обрабатывали в лаб.2. Вам потребуется в этом блокноте выполнить шаги 1 -3 лаб 2.

Далее выполнить (ранее это также было выполнено на шаге 11 в лаб.2) :

In [64]:
doc = nlp(aiwinter)
doc

кажется, что сейчас мы переживаем «золотой век» искусственного интеллекта. компании инвестируют десятки миллиардов долларов в разработку инструментов на базе генеративного ии. журналисты и исследователи экспериментируют с chatgpt, прося его то сочинить стих, то написать статью. правительства многих стран обсуждают или уже принимают законы, регулирующие использование алгоритмов. однако в истории уже были похожие периоды эйфории по поводу искусственного интеллекта. и заканчивались они разочарованием, оттоком инвестиций и замедлением развития технологий. рассказываем, почему происходили эти спады и стоит ли ожидать схожий итог у нынешней ии-лихорадки.«зима искусственного интеллекта» — период, когда происходит снижение финансирования и интереса к исследованиям в области ии. термин впервые появился в 1984 году как тема публичной дискуссии на ежегодной встрече американской ассоциации искусственного интеллекта (aaai). на ней ведущие исследователи в области ии роджер шэнк и марвин мински выска

In [65]:
nouns = [ token.text for token in doc if token.pos_ == 'NOUN' and token.is_punct !=True]

Создайте пары ключ-значение, под ключом имеется в виду слово из nouns, под значением - идентификатор. Сейчас всем словам назначается идентификатор 1

In [67]:
#создайте пустой список
lst = []



# создайте цикл. для каждого значения* из nouns сделать: новая переменная присвоить ей значение* + пробел + 1 под типом данных str
for noun in nouns:
    lst.append(f'{noun} 1')
# добавьте в список новая переменная

lst[0:2]
# результат должен быть таким ['c 1', 'a 1', 's 1', ...]




['век 1', 'интеллекта 1']

Выполните сортировку. Теперь мы сгруппируем вложенные структуры с одинаковыми ключами.

In [68]:
#примените к вашему списку функцию .sort()
# таким образом, результат буде ['a 1', 'c 1', 's 1', ...]
lst.sort()

 #### 3.2 Реализация функции Reduce

В этой части мы соберем все значения с одинаковым ключом, подсчитаем их количество и выдадим результат.

выполните dict.fromkeys(ваш список)

и примените к выражению выше конвертацию в list

In [69]:
dct = dict.fromkeys(lst, 0)

In [70]:
for key in lst:
    dct[key] += 1

In [71]:
result = sorted(dct.items(), key=lambda x: (-x[1], x[0]))

In [72]:
result

[('интеллекта 1', 5),
 ('области 1', 5),
 ('году 1', 4),
 ('исследования 1', 4),
 ('года 1', 3),
 ('исследователи 1', 3),
 ('ученые 1', 3),
 ('задач 1', 2),
 ('ии 1', 2),
 ('инвестиций 1', 2),
 ('интереса 1', 2),
 ('исследований 1', 2),
 ('период 1', 2),
 ('прогноз 1', 2),
 ('проектов 1', 2),
 ('сми 1', 2),
 ('термин 1', 2),
 ('технологий 1', 2),
 ('энтузиазм 1', 2),
 ('аккордом 1', 1),
 ('алгоритмов 1', 1),
 ('алгоритмы 1', 1),
 ('ассоциации 1', 1),
 ('базе 1', 1),
 ('век 1', 1),
 ('влияния 1', 1),
 ('встрече 1', 1),
 ('встречи 1', 1),
 ('год 1', 1),
 ('годы 1', 1),
 ('голову 1', 1),
 ('десятилетия 1', 1),
 ('десятки 1', 1),
 ('джорджтауне 1', 1),
 ('дискуссии 1', 1),
 ('долларов 1', 1),
 ('журналисты 1', 1),
 ('законы 1', 1),
 ('замедлением 1', 1),
 ('зима 1', 1),
 ('зимой 1', 1),
 ('инвесторы 1', 1),
 ('инициативой 1', 1),
 ('инструментов 1', 1),
 ('интеллект 1', 1),
 ('интерес 1', 1),
 ('информатик 1', 1),
 ('использование 1', 1),
 ('исследованиям 1', 1),
 ('иссяк 1', 1),
 ('истори